# Setting 1
## [MovieLens] KG

##### 이 파일에서 최종적으로, KGAT input txt 파일을 만든다.

##### 이 파일에서 모든 node들을 dict를 활용해서 mapping해줘서 모든 entity를 구분시킨다.

In [1]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
import os
import random
import numpy as np
from config import *
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import DBSCAN
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_score
import warnings
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from surprise import SVD, NMF, Dataset, Reader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
warnings.filterwarnings("ignore")

In [2]:
ml_movie = pd.read_csv("./preprocess_1/movie_0.csv")
c = pd.read_csv("./preprocess_1/cluster_0.csv")
d = pd.read_csv("./preprocess_1/director_0.csv")
i_s = pd.read_csv("./preprocess_1/item_sim_0.csv")
g = pd.read_csv("./preprocess_1/movie_genres_0.csv")
r = pd.read_csv("./preprocess_1/ratings_0.csv")
s = pd.read_csv("./preprocess_1/similarity_0.csv")
u_s = pd.read_csv("./preprocess_1/user_sim_0.csv")
user = pd.read_csv("./ml-100k/user.csv")

In [3]:
"""
TODO 1:
encoding: encoding dict, decoding dict 만들기
0. movie_id encoding
    - 0 ~ m
1. user_id encodeing
    - m ~ m + u
2. genre encoding
    - m + u ~ m + u + g
3. director encoding
    - m + u + g ~ m + u + g + d
4. storyline encoding
    - m + u + g + d  ~ m + u + g + d + s
5. cluster encoding
    - m + u + g + d + s  ~ m + u + g + d + s + c

TODO 2:
relation 지정하기
is_genre : 0
is_similar_user : 1
is_similar_item : 2
is_director : 3
is_storyline : 4
is_similar_storyline : 5
is_cluster : 6
is_prefer : 7

만들어야 하는 문자열 형식:
"entity_id_i relation_id entity_id_j"   (i != j)
"""
print()

In [4]:
ml_movie

,movie_id,title,release_date,video_release_date,IMDb_URL
0,1,toy story,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,goldeneye,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,four rooms,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,get shorty,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,copycat,01-Jan-95,01-Jan-95,http://us.imdb.com/M/title-exact?Copycat%20(1995)
...,...,...,...,...,...
1677,1678,mat' i syn,06-Feb-98,06-Feb-98,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...
1678,1679,b. monkey,06-Feb-98,06-Feb-98,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...
1679,1680,sliding doors,01-Jan-98,01-Jan-98,http://us.imdb.com/Title?Sliding+Doors+(1998)
1680,1681,you so crazy,01-Jan-94,01-Jan-94,http://us.imdb.com/M/title-exact?You%20So%20Cr...


In [5]:
total_storyline_id = set(list(s.movie_id.unique()) + list(s.similar_movie_id.unique()) + list(c.movie_id.unique()))

In [6]:
len(total_storyline_id)

402

In [7]:
total_storyline_named = [str(movie_id) + '_storyline' for movie_id in total_storyline_id]

In [8]:
storyline_naming_dict = {movie_id : new_name for movie_id, new_name in zip(total_storyline_id, total_storyline_named)}

별도로 storyline에 이름 부여

In [9]:
s['storyline_name'] = s.movie_id.map(storyline_naming_dict)
s['similar_storyline_name'] = s.similar_movie_id.map(storyline_naming_dict)

In [10]:
c['storyline_name'] = c.movie_id.map(storyline_naming_dict)

In [11]:
storyline_name_list = set(list(s.storyline_name.unique()) + list(s.similar_storyline_name.unique()) + list(c.storyline_name.unique()))

entity_list = [
    sorted(list(ml_movie.movie_id.unique())),
    sorted(list((user.user_id).unique())),
    sorted(list(g.genre.unique())),
    # sorted(list(d.director_name.unique())),
    # sorted(list(storyline_name_list)),
    # sorted(list(c.cluster_id.unique())),
]

In [12]:
"""
TODO 1:
encoding: encoding dict, decoding dict 만들기
0. movie_id encoding
    - 0 ~ m
1. user_id encodeing
    - m ~ m + u
2. genre encoding
    - m + u ~ m + u + g
3. director encoding
    - m + u + g ~ m + u + g + d
4. storyline encoding
    - m + u + g + d  ~ m + u + g + d + s
5. cluster encoding
    - m + u + g + d + s  ~ m + u + g + d + s + c
"""

entity_lists = []
entity_len = []
encoding_list = []
decoding_list = []

for i in range(len(entity_list)):
    entity_lists.append(entity_list[i])
    entity_len.append(len(entity_lists[i]))
    encoding_list.append({v : int(idx + sum(entity_len[:i])) for idx, v in enumerate(entity_lists[i])})
    decoding_list.append({idx : v for v, idx in encoding_list[i].items()})

In [13]:
for i in range(len(entity_list)):
    print(sum(entity_len[:i]))

0
1682
2625


In [14]:
g.head(1)

,genre_id,movie_id,genre
0,1,267,unknown


In [15]:
c.head(1)

,title,overview,cluster_id,movie_id,storyline_name
0,titanic,"84 years later, a 101-year-old woman named Ros...",13,313,313_storyline


In [16]:
s.head(1)

,title,cosine_similarity,movie_id,similar_movie_id,storyline_name,similar_storyline_name
0,mars attacks!,0.852736,257,235,257_storyline,235_storyline


In [17]:
"""
TODO 2:
relation 지정하기
is_genre : 0
is_similar_user : 1
is_similar_item : 2
is_director : 3
is_storyline : 4
is_similar_storyline : 5
is_cluster : 6

만들어야 하는 문자열 형식:
"entity_id_i relation_id entity_id_j"   (i != j)
"""
relations = []

# is_genre : 0
g_ = g[["movie_id", "genre"]].drop_duplicates()
g_.insert(1, 'relation', 0)
g_.movie_id = g_.movie_id.map(encoding_list[0])
g_.genre = g_.genre.map(encoding_list[2])

# # is_similar_user : 1
# u_s_ = u_s[["user_id", "similar_user_id"]].drop_duplicates()
# u_s_.insert(1, 'relation', 1)
# u_s_.user_id = u_s_.user_id.map(encoding_list[1])
# u_s_.similar_user_id = u_s_.similar_user_id.map(encoding_list[1])

# # is_similar_item : 2
# i_s_ = i_s[["item_id", "similar_item_id"]].drop_duplicates()
# i_s_.insert(1, 'relation', 2)
# i_s_.item_id = i_s_.item_id.map(encoding_list[0])
# i_s_.similar_item_id = i_s_.similar_item_id.map(encoding_list[0])

# # is_director : 3
# d_ = d[["movie_id", "director_name"]].drop_duplicates()
# d_.insert(1, 'relation', 3)
# d_.movie_id = d_.movie_id.map(encoding_list[0])
# d_.director_name = d_.director_name.map(encoding_list[3])

# # is_storyline : 4
# # storyline_name에 대한 모든 storyline_name 취합하기
# cc = c[["movie_id", "storyline_name"]]
# ss = s[["movie_id", "storyline_name"]]
# sss = s[["similar_movie_id", "similar_storyline_name"]]
# sss.rename(columns={"similar_movie_id":"movie_id"}, inplace=True)
# sss.rename(columns={"similar_storyline_name":"storyline_name"}, inplace=True)
# total_storyline_relation = pd.concat([cc, ss, sss], axis=0)

# tsr_ = total_storyline_relation.drop_duplicates()
# tsr_.insert(1, 'relation', 4)
# tsr_.movie_id = tsr_.movie_id.map(encoding_list[0])
# tsr_.storyline_name = tsr_.storyline_name.map(encoding_list[4])

# # is_similar_storyline : 5
# s_ = s[["storyline_name", "similar_storyline_name"]].drop_duplicates()
# s_.insert(1, 'relation', 5)
# s_.storyline_name = s_.storyline_name.map(encoding_list[4])
# s_.similar_storyline_name = s_.similar_storyline_name.map(encoding_list[4])

# # is_cluster : 6
# c_ = c[["movie_id", "cluster_id"]].drop_duplicates()
# c_.insert(1, 'relation', 6)
# c_.cluster_id = c_.cluster_id.map(encoding_list[5])
# c_.movie_id = c_.movie_id.map(encoding_list[0])

In [18]:
# 모든 column을 'head', 'relation', 'tail'로 변경
g_.rename(columns={"movie_id":"head"}, inplace=True)
g_.rename(columns={"genre":"tail"}, inplace=True)

# u_s_.rename(columns={"user_id":"head"}, inplace=True)
# u_s_.rename(columns={"similar_user_id":"tail"}, inplace=True)

# i_s_.rename(columns={"item_id":"head"}, inplace=True)
# i_s_.rename(columns={"similar_item_id":"tail"}, inplace=True)

# d_.rename(columns={"movie_id":"head"}, inplace=True)
# d_.rename(columns={"director_name":"tail"}, inplace=True)

# tsr_.rename(columns={"movie_id":"head"}, inplace=True)
# tsr_.rename(columns={"storyline_name":"tail"}, inplace=True)

# s_.rename(columns={"storyline_name":"head"}, inplace=True)
# s_.rename(columns={"similar_storyline_name":"tail"}, inplace=True)

# c_.rename(columns={"movie_id":"head"}, inplace=True)
# c_.rename(columns={"cluster_id":"tail"}, inplace=True)

In [19]:
# 모든 relation을 하나의 KG로 통합
kg = pd.concat([g_], axis=0)

In [20]:
kg.to_csv("./setting_1/kg_final.csv", index=False, sep=" ")

In [46]:
"""
TODO 1:
encoding: encoding dict, decoding dict 만들기
0. movie_id encoding
    - 0 ~ m
1. user_id encodeing
    - m ~ m + u
2. genre encoding
    - m + u ~ m + u + g
3. director encoding
    - m + u + g ~ m + u + g + d
4. storyline encoding
    - m + u + g + d  ~ m + u + g + d + s
5. cluster encoding
    - m + u + g + d + s  ~ m + u + g + d + s + c
"""
print()

In [49]:
def split_by_user(data, split_ratio:float=0.9):
    # user_id별로 데이터를 분리하여 train, test dataset으로 나누기
    train = pd.DataFrame()
    test = pd.DataFrame()
    if split_ratio == 1.0:
        for user_id in tqdm(data['user_id'].unique()):
            user_df = data[data['user_id'] == user_id]
            train = pd.concat([train, user_df], ignore_index=True)
        return train, None
    else:
        for user_id in tqdm(data['user_id'].unique()):
            user_df = data[data['user_id'] == user_id]
            train_df, test_df = train_test_split(user_df, test_size=split_ratio, random_state=42)
            train = pd.concat([train, train_df], ignore_index=True)
            test = pd.concat([test, test_df], ignore_index=True)
        return train, test

In [56]:
test, train = split_by_user(r[r.rating >= 4].groupby('user_id').filter(lambda x: len(x) >= 10))

100%|██████████| 897/897 [00:03<00:00, 293.70it/s]


In [57]:
print(len(train), len(test))

49938 5111


In [58]:
train.head(1)

,rating_id,user_id,movie_id,rating,timestamp
0,26560,298,23,4,1998-01-07 14:27


In [61]:
train = train.sort_values(by=['user_id', 'movie_id', 'rating'])
test = test.sort_values(by=['user_id', 'movie_id', 'rating'])

In [89]:
train_ = train.groupby('user_id')["movie_id"].apply(lambda x: ''.join(str(sorted(x)).strip("[").strip("]").split(","))).reset_index()
test_ = test.groupby('user_id')["movie_id"].apply(lambda x: ''.join(str(sorted(x)).strip("[").strip("]").split(","))).reset_index()

In [90]:
train_data, test_data = [], []

for row in train_.iterrows():
    train_data.append(str(row[0]) + " " + str(row[1].movie_id))

for row in test_.iterrows():
    test_data.append(str(row[0]) + " " + str(row[1].movie_id))

In [91]:
with open("./kg/train_data.txt", "w") as output:
    output.write("\n".join(train_data))

In [92]:
with open("./kg/test_data.txt", "w") as output:
    output.write("\n".join(test_data))